In [11]:
import json
file = json.load(open("gg2013answers.json"))
data = file['award_data']
data

{'best screenplay - motion picture': {'nominees': ['zero dark thirty',
   'lincoln',
   'silver linings playbook',
   'argo'],
  'presenters': ['robert pattinson', 'amanda seyfried'],
  'winner': 'django unchained'},
 'best director - motion picture': {'nominees': ['kathryn bigelow',
   'ang lee',
   'steven spielberg',
   'quentin tarantino'],
  'presenters': ['halle berry'],
  'winner': 'ben affleck'},
 'best performance by an actress in a television series - comedy or musical': {'nominees': ['zooey deschanel',
   'tina fey',
   'julia louis-dreyfus',
   'amy poehler'],
  'presenters': ['aziz ansari', 'jason bateman'],
  'winner': 'lena dunham'},
 'best foreign language film': {'nominees': ['the intouchables',
   'kon tiki',
   'a royal affair',
   'rust and bone'],
  'presenters': ['arnold schwarzenegger', 'sylvester stallone'],
  'winner': 'amour'},
 'best performance by an actor in a supporting role in a motion picture': {'nominees': ['alan arkin',
   'leonardo dicaprio',
   'phil

In [15]:
for key in data.keys():
    print(key)

best screenplay - motion picture
best director - motion picture
best performance by an actress in a television series - comedy or musical
best foreign language film
best performance by an actor in a supporting role in a motion picture
best performance by an actress in a supporting role in a series, mini-series or motion picture made for television
best motion picture - comedy or musical
best performance by an actress in a motion picture - comedy or musical
best mini-series or motion picture made for television
best original score - motion picture
best performance by an actress in a television series - drama
best performance by an actress in a motion picture - drama
cecil b. demille award
best performance by an actor in a motion picture - comedy or musical
best motion picture - drama
best performance by an actor in a supporting role in a series, mini-series or motion picture made for television
best performance by an actress in a supporting role in a motion picture
best television serie